In [1]:
import src.game_runners as gr
import pickle
import numpy as np
import matplotlib.pyplot as plt
number_of_experiments = 10

2024-02-21 18:36:45.316845: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-21 18:36:45.359200: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 18:36:45.359234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 18:36:45.360325: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-21 18:36:45.367640: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-21 18:36:45.368100: I tensorflow/core/platform/cpu_feature_guard.cc:1

Instructions for updating:
non-resource variables are not supported in the long term
tf_ver:2.15.0
tf_ver:2.15.0


In [2]:
# Run the experiments for actor critic with bigger net 
policy_actor_critic_results_big_net = []
for i in range(number_of_experiments):
    policy_actor_critic_results_big_net.append(gr.run_actor_critic(is_big=True))
    
with open('result_score/policy_actor_critic_results_big_net.pkl', 'wb') as f:
    pickle.dump(policy_actor_critic_results_big_net, f)


2024-02-21 18:36:50.301825: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-02-21 18:36:50.308270: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
 28%|██▊       | 1422/5000 [11:31<29:00,  2.06it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../result_score/policy_actor_critic_results_big_net.pkl'

In [8]:
# Run the experiments for basic REINFORCE
policy_results = []
for i in range(number_of_experiments):
    policy_results.append(gr.run())
    
with open('result_score/policy_results.pkl', 'wb') as f:
    pickle.dump(policy_results, f)

 19%|█▉        | 960/5000 [02:54<12:14,  5.50it/s]


UnsupportedOperation: write

In [10]:
# Run the experiments for REINFORCE with baseline
policy_baseline_results = []
for i in range(number_of_experiments):
    policy_baseline_results.append(gr.run_with_baseline())
    
with open('result_score/policy_baseline_results.pkl', 'wb') as f:
    pickle.dump(policy_baseline_results, f)    

 11%|█         | 531/5000 [02:40<22:26,  3.32it/s]


In [11]:
# Run the experiments for actor critic with smaller net 
policy_actor_critic_results = []
for i in range(number_of_experiments):
    policy_actor_critic_results.append(gr.run_actor_critic())
    
with open('result_score/policy_actor_critic_results.pkl', 'wb') as f:
    pickle.dump(policy_actor_critic_results, f)

 43%|████▎     | 2150/5000 [05:38<07:28,  6.35it/s]
